In [1]:
# from randtest import randtest
import numpy as np
import pandas
import pickle
from tqdm import tqdm
import random
import tensorflow.keras.backend as K

2024-06-21 17:24:45.695890: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 17:24:46.589954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 17:25:03.455283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
print(cores)

80


In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
def forecast_loss(y_true, y_pred):
    V=134
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

In [5]:
def score(y_preds, y_true):
    y_pred = y_preds.reshape(-1, 134)
    mmse = forecast_loss(y_true, y_pred)
    s = 0
    for i in mmse:
        s += i
    return s/len(mmse)

In [6]:
def ra_test(a_0, b_0, y_true, r=1000):
    """randomization test"""
    if len(a_0) != len(b_0):
        print('Input A and B are not eaqul-sized!')
        return
    delta_s_rs = []
    c = 0
    n = len(a_0)
    
    
    delta_s_0 = score(a_0, y_true) - score(b_0, y_true)
    delta_s_0 = delta_s_0.numpy()
    
    for _ in tqdm(range(r)):
        a_r = []
        b_r = []
        # shuffle
        for i in range(n):
            # swap
            rn = random.randint(0, 1)
            # swap
            if rn == 0:
                a_r.append(b_0[i])
                b_r.append(a_0[i])
            # do not swap
            else:
                a_r.append(a_0[i])
                b_r.append(b_0[i])

        a_r = np.array(a_r)
        b_r = np.array(b_r)
        delta_s_r = score(a_r, y_true) - score(b_r, y_true)
        delta_s_r = delta_s_r.numpy()
        # print(delta_s_r)
        # print(abs(delta_s_r))
        delta_s_rs.append(delta_s_r)
        if abs(delta_s_r) >= abs(delta_s_0):
            c += 1
        
    print(c)
    p = c/r
    print(delta_s_rs)
    return p

In [7]:
baseline_path = 'randomization_test/data/baseline.pkl'
baseline_data, var_to_ind = pickle.load(open(baseline_path, 'rb'))

forecasting_preds = []

# y_true for score
y_true = []
for y in baseline_data['forecasting_test_op']:
    y_true.append(y)
y_true = np.array(y_true)

for pred in baseline_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)
x = forecasting_preds.flatten()
del baseline_data, forecasting_preds

In [8]:
m_path = 'randomization_test/data/hierar.pkl'
m_data, var_to_ind_m = pickle.load(open(m_path, 'rb'))
m_data

forecasting_preds = []

for pred in m_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)

y = forecasting_preds.flatten()
del m_data, forecasting_preds

In [9]:
p = ra_test(x, y, y_true = y_true, r=100)
p

2024-06-21 17:25:56.224260: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31117 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b3:00.0, compute capability: 7.0
100%|██████████| 100/100 [2:14:07<00:00, 80.47s/it] 

0
[-0.0005296424972112135, -0.004553900039431902, 0.012495103937136598, 0.0015157986719271932, 0.009236292862967233, -0.002315424827965984, 0.0074941801507177885, 0.005425533181920628, -0.0005425688003040108, 0.0019975485997196785, -0.0066643002598487655, -0.009133231081256632, 0.013586781735019038, 0.011345473631124925, 0.00409072180883463, 0.0032712873878972815, -0.004158963775303448, 0.0012143450526052035, 0.00911151684922995, -0.005116800565244439, 0.013072552907154034, -0.0013491261520188758, -0.001997769760747836, 0.00379482049962121, -0.013967458635134378, 0.012535788302572293, 0.01092551213956039, 0.01199624148790246, -0.004733348129662751, -0.010049195501308006, -0.0061817325752446806, 0.00031259987862597427, -0.0014653018506995963, -0.014464713664316342, -0.0014541889235397676, 0.0091979445358783, 0.001119274264635095, -0.020519337039632823, 0.02361658750036888, 0.0034756999208580552, -0.0009465341432299823, 0.005356383658022601, -0.007256834842105597, 0.010163952385766173, 0

0.0

In [13]:
p

0.0

In [14]:
p

0.0

In [15]:
p

0.0